## **IMPORT LIBRARIES**

In [1]:
import pandas as pd

## **DEFAULT FUNCTIONS**

In [2]:
def read_csv(file_name: str) -> pd.DataFrame:
    path = './test_data/' + file_name + '.csv'

    data = pd.read_csv(path, sep=',')

    return data

In [3]:
def norm_money(money: str):
    return int(money.replace(' ₫', '').replace(',', ''))

In [4]:
def norm_route(cr: list[int]):
    route = [-1]

    for r in cr:
        if route[len(route)-1] == r:
            continue
        else:
            route.append(r)

    return route[1:]

In [5]:
def create_route(cr: list[int]):    
    str_route = []

    for i in range(len(cr)-1):
        str_route.append(str(cr[i]) + ' to ' + str(cr[i+1]))

    return str_route

## **SOLUTIONS**

### Question 1

In [6]:
def q1(data: pd.DataFrame, prod_info: pd.DataFrame):
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'

    lns = data.loc[:,['Shelf ID', 'Item ID', 'Looking at item (s)', 'Holding the item (s)']]

    lns['Sum'] = lns['Looking at item (s)'] + lns['Holding the item (s)']
    lns['Name'] = lns.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)

    grouped_lns = lns.groupby('Name')['Sum'].sum()

    return grouped_lns.sort_values(ascending=False)

### Question 2

In [7]:
def q2(data: pd.DataFrame, prod_info: pd.DataFrame):
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    hnr = data.loc[:,['Shelf ID', 'Item ID', 'Holding the item (s)', 'Returning item']]
    hnr.dropna()

    hnr['Name'] = hnr.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)

    hnr_true = hnr[hnr['Returning item'] == True]

    return hnr_true['Name'].value_counts(ascending=False)      
    

### Question 3

In [8]:
def q3(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
                
    anp = data.loc[:,['Shelf ID', 'Item ID', 'Age', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp['Name'] = anp.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1) 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    print(anp_buy.size, anp.size)

    aba_thn = anp_buy[(anp_buy['Age'] >= 18) & (anp_buy['Age'] <= 30)]
    aba_trn = anp_buy[(anp_buy['Age'] >= 31) & (anp_buy['Age'] <= 60)]
    aba_cn = anp_buy[(anp_buy['Age'] >= 61)]

    return aba_thn['Name'].value_counts(ascending=False), aba_trn['Name'].value_counts(ascending=False), aba_cn['Name'].value_counts(ascending=False) 
    

### Question 4

In [9]:
def q4(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_price(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return norm_money(row.iloc[0]['Price'])
        except:
            return 'Not Found!'
 
    anp = data.loc[:,['Shelf ID', 'Item ID', 'Timestamp', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1) 
    anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1) 

    anp_buy['Timestamp'] = pd.to_datetime(anp_buy['Timestamp'], unit='s')
    anp_buy['Date'] = anp_buy['Timestamp'].dt.date

    return anp_buy.groupby('Date')['Price'].sum().sort_values(ascending=False)


### Question 5

In [10]:
def q5(data: pd.DataFrame):                  
    anp = data.loc[:,['Age']]

    anp_thn = anp[(anp['Age'] >= 18) & (anp['Age'] <= 30)]
    anp_trn = anp[(anp['Age'] >= 31) & (anp['Age'] <= 60)]
    anp_cn = anp[(anp['Age'] >= 61)]

    return anp_thn.size, anp_trn.size, anp_cn.size
    

### Question 6

In [11]:
def q6(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_sale(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Discount']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Sale'] = prods_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)   

    pbs = prods_buy[prods_buy['Sale'] != 0]

    return pbs.groupby('Name')['Name'].value_counts().sort_values(ascending=False)
    

### Question 7

In [12]:
def q7(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_strategy(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Marketing strategy']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Strategy'] = prods_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)   

    pbs = prods_buy[prods_buy['Strategy'] == True]

    print(pbs['Strategy'])

    return pbs.groupby('Name')['Name'].value_counts().sort_values(ascending=False)
    

### Question 8

In [13]:
def q8(data: pd.DataFrame):
    lns = data.loc[:,['Shelf ID', 'Item ID', 'Looking at item (s)', 'Holding the item (s)']]

    lns['Sum'] = lns['Looking at item (s)'] + lns['Holding the item (s)']
    
    result = lns.groupby('Shelf ID').agg(
        total_time = ('Sum', 'sum'),
        interaction = ('Shelf ID', 'count')
    )

    result['avg'] = result['total_time'] / result['interaction']

    return result.sort_values(by=['avg'], ascending=False)

### Question 9

In [14]:
def q9(data: pd.DataFrame):          
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    return prods_buy.groupby('Shelf ID')['Shelf ID'].value_counts().sort_values(ascending=False)
    

### Question 10

In [15]:
def q10(data: pd.DataFrame):          
    cust = list(data.loc[:,['Person ID']]['Person ID'].unique())
    routes = []

    for cust_id in cust:
        cr = norm_route(list(data[(data['Person ID'] == cust_id)]['Shelf ID']))
        if len(cr) != 1:
            routes += create_route(cr)

    rdf = pd.DataFrame(routes, columns=['Route'])

    return rdf.value_counts().sort_values(ascending=False)    

### Question 11

In [35]:
def q11_1(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_price(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return norm_money(row.iloc[0]['Price'])
        except:
            return 'Not Found!'
 
    anp = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1) 

    return anp_buy.groupby('Shelf ID')['Price'].sum().sort_values(ascending=False)


In [60]:
def q11_2(data: pd.DataFrame, prod_info: pd.DataFrame, shelf: int):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
    
    def get_prod_price(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return norm_money(row.iloc[0]['Price'])
        except:
            return 'Not Found!'
 
    def get_prod_location(name):
        try:
            row = prod_info[(prod_info['Name'] == name)]            
            return row.iloc[0]['Location']
        except:
            return 'Not Found!'

    anp = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1) 
    anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1)

    result = anp_buy[anp_buy['Shelf ID'] == shelf]

    grouped_result = result.groupby('Name').agg(
        total = ('Price', 'sum')
    ).reset_index()

    grouped_result['Location'] = grouped_result.apply(lambda row: get_prod_location(row['Name']), axis=1)

    return grouped_result.sort_values(by=['total'], ascending=False)


### Question 12

In [113]:
def q12_1(data: pd.DataFrame, prod_info: pd.DataFrame, shelf: int):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_sale(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Discount']
        except:
            return 'Not Found!'
        
    def get_prod_strategy(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Marketing strategy']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna()

    pbs = prods[prods['Shelf ID'] == shelf] 

    pbs_buy = pbs[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    pbs_buy['Name'] = pbs_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    pbs_buy['Sale'] = pbs_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)   
    pbs_buy['Strategy'] = pbs_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)  

    both = pbs_buy[(pbs_buy['Strategy'] == True) & ((pbs_buy['Sale'] != 0))]['Name'].size
    marketing = pbs_buy[(pbs_buy['Strategy'] == True) & ((pbs_buy['Sale'] == 0))]['Name'].size 
    sales = pbs_buy[(pbs_buy['Strategy'] == False) & ((pbs_buy['Sale'] != 0))]['Name'].size 
    nothing = pbs_buy[(pbs_buy['Strategy'] == False) & ((pbs_buy['Sale'] == 0))]['Name'].size

    return both, marketing, sales, nothing
    

In [114]:
def q12_2(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_sale(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Discount']
        except:
            return 'Not Found!'
        
    def get_prod_strategy(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Marketing strategy']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna()

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Sale'] = prods_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Strategy'] = prods_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)  

    both = prods_buy[(prods_buy['Strategy'] == True) & ((prods_buy['Sale'] != 0))]['Name'].size
    marketing = prods_buy[(prods_buy['Strategy'] == True) & ((prods_buy['Sale'] == 0))]['Name'].size 
    sales = prods_buy[(prods_buy['Strategy'] == False) & ((prods_buy['Sale'] != 0))]['Name'].size 
    nothing = prods_buy[(prods_buy['Strategy'] == False) & ((prods_buy['Sale'] == 0))]['Name'].size

    return both, marketing, sales, nothing
    

## **RUN IT!**

### Init

In [17]:
cust_data = read_csv('03_Customer_Behavior_Data')
item_data = read_csv('03_Item_Information_Data')
shelf_data = read_csv('03_Shelf_Information_Data')

In [18]:
cust_data.groupby('Shelf ID')['Shelf ID'].value_counts().sort_values(ascending=False)

Shelf ID
7    3305
0    2789
1    2363
2    1909
4    1878
3    1302
6    1087
5     761
Name: count, dtype: int64

In [19]:
cust_data.groupby('Shelf ID')['Looking at item (s)'].sum()

Shelf ID
0     78503
1     61420
2     54811
3     40689
4     57541
5     24391
6     23008
7    101444
Name: Looking at item (s), dtype: int64

### Answer 1

In [20]:
a1 = q1(cust_data, item_data)

a1

Name
Sữa chua uống Probi                                                    22896
Sữa chua uống Yakult                                                   22896
Sữa ông thọ                                                            13939
Bim bim Oishi                                                          13866
Snack khoai tây Lays                                                   13362
                                                                       ...  
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)     1669
Sách Thám tử Sherlock Home                                              1463
Sách cho trẻ từ 1-2 tuổi                                                1421
Truyện Doraemon                                                         1378
Sách Ðạo Tình                                                           1152
Name: Sum, Length: 134, dtype: int64

### Answer 2

In [21]:
a2 = q2(cust_data, item_data)

a2

Name
4 hộp sữa lúa mạch Milo 180ml    134
Snack khoai tây Lays             127
Mý ý SG Food                     117
Nước lẩu Barona                  116
Sữa chua Vinamik                 114
                                ... 
Bia Sài Gòn Special                8
Chảo Lock&Lock                     7
Bếp nướng Lock&Lock                7
Chảo Sunhouse                      6
Máy xay sinh tố Comet              6
Name: count, Length: 131, dtype: int64

### Answer 3

In [22]:
a3_thn, a3_trn, a3_cn = q3(cust_data, item_data)

41440 107758


In [23]:
a3_thn

Name
Bánh trứng Custard                            46
Lốc 4 hộp sữa tươi ít đường TH true MILK      45
Sữa bột Milo                                  38
Tã dán Merries size NB 76 miếng (dưới 5kg)    32
Kem tràng tiền                                31
                                              ..
Miếng dong Miến vương                          1
Sách cho trẻ sơ sinh đến 6 tháng tuổi          1
Ngũ cốc VinaCafe                               1
Nồi cơm điện Kangaroo                          1
Sách cho trẻ từ 1-2 tuổi                       1
Name: count, Length: 132, dtype: int64

In [24]:
a3_trn

Name
Kem tràng tiền                                                         96
Lốc 4 hộp sữa tươi ít đường TH true MILK                               92
Sữa bột Milo                                                           78
Bánh trứng Custard                                                     72
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)    66
                                                                       ..
Ðèn bàn Ðiện Quang                                                      3
Sách cho trẻ từ 4-5 tuổi                                                2
Nồi cơm điện Kangaroo                                                   1
Sách cho trẻ từ 2-3 tuổi                                                1
Sách Thám tử Sherlock Home                                              1
Name: count, Length: 134, dtype: int64

In [25]:
a3_cn

Name
Bánh trứng Custard                            53
Sữa bột Milo                                  51
Lốc 4 hộp sữa tươi ít đường TH true MILK      50
Kem tràng tiền                                42
Tã dán Merries size NB 76 miếng (dưới 5kg)    38
                                              ..
Phích cắm điện Ðiện Quang                      1
Bếp điện từ Sunhouse                           1
Sách cho trẻ từ 4-5 tuổi                       1
Sách cho trẻ sơ sinh đến 6 tháng tuổi          1
Xì dầu Tam Thái tử                             1
Name: count, Length: 134, dtype: int64

### Answer 4

In [26]:
a4 = q4(cust_data, item_data)

a4

C:\Users\HP\AppData\Local\Temp\ipykernel_13040\3707479508.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_13040\3707479508.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_13040\3707479508.py:30: SettingWithCopyWarning:

Date
2024-07-06    238686900
2024-07-07    221067200
2024-07-02     69922100
2024-07-04     68094500
2024-07-05     62646400
2024-07-03     51912100
2024-07-08     29413400
2024-07-01     19623300
Name: Price, dtype: int64

### Answer 5

In [27]:
a3_thn, a3_trn, a3_cn = q5(cust_data)

print(a3_thn, a3_trn, a3_cn)

3574 7617 4203


### Answer 6

In [28]:
a6 = q6(cust_data, item_data)

a6

C:\Users\HP\AppData\Local\Temp\ipykernel_13040\2522391787.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_13040\2522391787.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Sale'] = prods_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)


Name
Bánh trứng Custard                                171
Kem tràng tiền                                    169
Sữa bột Milo                                      167
Dầu gội Romano                                    105
Khăn mặt Shine                                    102
Khăn tắm Shine                                     98
Tã dán sơ sinh Goo.n Premium Newborn 70 miếng     88
Dầu gội Dove                                       87
Hạt hướng dương Chacheer                           78
Nước mắm Nam ngư                                   77
Snack khoai tây Lays                               76
Nước mắm cá cơm Thuận Phát                         75
Bim bim Oishi                                      73
Sữa chua uống Probi                                69
Socola trứng Kinder                                66
Ðường kính Toàn Phát                               66
Nước mắm hạnh phúc                                 64
Sửa bột Anele                                      62
Bánh Oreo              

### Answer 7

In [29]:
a7 = q7(cust_data, item_data)

a7

C:\Users\HP\AppData\Local\Temp\ipykernel_13040\1486830990.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)


21       True
23       True
40       True
45       True
53       True
         ... 
15327    True
15350    True
15355    True
15363    True
15365    True
Name: Strategy, Length: 1211, dtype: bool


C:\Users\HP\AppData\Local\Temp\ipykernel_13040\1486830990.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Strategy'] = prods_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)


Name
Bánh trứng Custard                                171
Kem tràng tiền                                    169
Sữa bột Milo                                      167
Khăn mặt Shine                                    102
Khăn tắm Shine                                     98
Tã dán sơ sinh Goo.n Premium Newborn 70 miếng     88
Dầu gội Dove                                       87
Nước mắm cá cơm Thuận Phát                         75
Strongbow dâu đen                                  36
Bếp điện từ Lock&Lock                              35
Máy xay sinh tố Lock&Lock                          34
Chảo Lock&Lock                                     33
Gạo ST25 Neptune                                   31
Nồi cơm điện Cuckcoo                               30
Nồi cơm điện Lock&Lock                             30
Bếp nướng Lock&Lock                                25
Name: count, dtype: int64

### Answer 8

In [30]:
a8 = q8(cust_data)

a8

,total_time,interaction,avg
Shelf ID,,,
5,47380,761,62.260184
7,201638,3305,61.009985
3,77919,1302,59.845622
4,108934,1878,58.005325
2,106461,1909,55.767941
0,153458,2789,55.022589
1,120088,2363,50.820144
6,42914,1087,39.479301


### Answer 9

In [31]:
a9 = q9(cust_data)

a9

Shelf ID
1    1158
7    1146
0     969
2     774
4     698
3     522
5     371
6     282
Name: count, dtype: int64

### Answer 10

In [32]:
a10 = q10(cust_data)

a10

Route 
7 to 0    123
0 to 7    109
1 to 7    104
7 to 4     92
7 to 1     92
7 to 2     79
2 to 0     77
4 to 7     77
2 to 7     76
0 to 1     72
1 to 0     71
4 to 1     65
0 to 2     65
1 to 2     59
0 to 3     57
1 to 4     57
2 to 1     56
3 to 7     54
4 to 0     54
0 to 4     54
3 to 0     49
2 to 4     48
0 to 6     46
4 to 2     43
7 to 3     40
5 to 7     38
7 to 6     37
3 to 1     36
7 to 5     35
6 to 0     35
4 to 3     34
1 to 6     33
5 to 1     32
1 to 3     32
6 to 1     32
2 to 3     31
6 to 7     31
3 to 2     31
6 to 4     28
6 to 2     27
4 to 6     26
1 to 5     26
5 to 0     26
3 to 4     25
2 to 6     24
5 to 4     20
2 to 5     19
0 to 5     19
5 to 2     19
3 to 6     17
4 to 5     16
3 to 5     15
5 to 3     14
6 to 3     13
6 to 5     11
5 to 6      9
Name: count, dtype: int64

### Answer 11

In [33]:
a11_1 = q11_1(cust_data, item_data)

a11_1

C:\Users\HP\AppData\Local\Temp\ipykernel_13040\1699850882.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1)


Shelf ID
5    292315000
1    175475000
2    130884000
7     53623900
4     32642800
6     29129000
0     24424700
3     22871500
Name: Price, dtype: int64

In [34]:
a8

,total_time,interaction,avg
Shelf ID,,,
5,47380,761,62.260184
7,201638,3305,61.009985
3,77919,1302,59.845622
4,108934,1878,58.005325
2,106461,1909,55.767941
0,153458,2789,55.022589
1,120088,2363,50.820144
6,42914,1087,39.479301


In [71]:
thuc_pham = q11_2(cust_data, item_data, 3)
gia_vi = q11_2(cust_data, item_data, 4)

C:\Users\HP\AppData\Local\Temp\ipykernel_13040\2594595495.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_13040\2594595495.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_13040\2594595495.py:34: SettingWithCopyWarning:

In [81]:
thuc_pham.groupby('Location')['total'].sum()

Location
Cao            2412500
Thấp           4696200
Trung bình    15762800
Name: total, dtype: int64

In [82]:
thuc_pham.groupby('Location')['Name'].count()

Location
Cao            3
Thấp           4
Trung bình    18
Name: Name, dtype: int64

In [95]:
thuc_pham.sort_values(by=['Location', 'total'], ascending=False)

,Name,total,Location
22,Vodka Hà Nội,4920000,Trung bình
21,Vodka Cá sấu,1820000,Trung bình
11,Mỳ Omachi,1584000,Trung bình
20,Vang đỏ đà lạt,1560000,Trung bình
1,Bia Sài Gòn Special,848000,Trung bình
16,Phở khô Chinsu,805000,Trung bình
19,Strongbow dâu đen,691200,Trung bình
2,Bánh đa Vifon,690000,Trung bình
8,Miếng dong Miến vương,648000,Trung bình
14,Nui Safoco,490000,Trung bình


In [80]:
gia_vi.groupby('Location')['total'].sum()

Location
Cao           24342000
Thấp           4176200
Trung bình     4124600
Name: total, dtype: int64

In [84]:
gia_vi.groupby('Location')['Name'].count()

Location
Cao           7
Thấp          5
Trung bình    7
Name: Name, dtype: int64

In [94]:
gia_vi.sort_values(by=['total'], ascending=False)

,Name,total,Location
12,Nước mắm hạnh phúc,9280000,Cao
11,Nước mắm cá cơm Thuận Phát,4425000,Cao
8,Nước mắm Nam ngư,4350500,Cao
4,Dầu ăn Tường An,1999500,Cao
3,Dầu ăn Simply,1896000,Cao
7,Nước mắm Chinsu,1404000,Cao
18,Ðường phèn Biên Hoà,1278000,Thấp
5,Mỳ chính Ajinomoto,1008000,Thấp
2,Dầu ăn Meizan,987000,Cao
1,Dầu hào Maggi,969000,Trung bình


### Answer 12

In [ ]:
bc_banh_keo = q12_1(cust_data, item_data, 0)
bc_hmp = q12_1(cust_data, item_data, 1)
bc_sua = q12_1(cust_data, item_data, 2)
bc_thuc_pham = q12_1(cust_data, item_data, 3)
bc_gia_vi = q12_1(cust_data, item_data, 4)
bc_gia_dung = q12_1(cust_data, item_data, 5)
bc_sdc = q12_1(cust_data, item_data, 6)
bc_dong_lanh = q12_1(cust_data, item_data, 7)

In [ ]:
bc_all = q12_2(cust_data, item_data)

In [121]:
bc_all

(1211, 0, 1687, 3022)

In [117]:
bc_banh_keo

(171, 0, 407, 391)

In [118]:
bc_hmp

(375, 0, 288, 495)

In [122]:
bc_sua

(167, 0, 62, 545)

In [123]:
bc_thuc_pham

(67, 0, 214, 241)

In [124]:
bc_gia_vi

(75, 0, 463, 160)

In [125]:
bc_gia_dung

(187, 0, 184, 0)

In [126]:
bc_sdc

(0, 0, 0, 282)

In [127]:
bc_dong_lanh

(169, 0, 69, 908)

## **IMPORTANT NOTE**

### **0. Về dự án**

Cảm ơn cuộc thi DSTC 2024 đã mang lại cho mình một cơ hội tốt để áp dụng các kiến thức đã học vào một trường hợp cụ thể trong thực tế!

Trên đây là phần trình bày về cách thức xử lý dữ liệu để đi đến đáp án cho từng câu hỏi trong đề bài!

Vì yêu cầu là chỉ gửi về file xử lý dữ liệu dưới dạng .zip nên mình sẽ không đính kèm các file dữ liệu mà Ban tổ chức (BTC) đã gửi vào đây! Nếu có nhu cầu kiểm thử lại quá trình này, các bạn có thể truy cập vào repository và tải về thông qua đường dẫn: https://github.com/duy-nq/FTU-Mock-Test


### **1. Về logic mua hàng**

Sau khi quan sát một vài dòng dữ liệu trong bộ dataset liên quan đến hành vi mua sắm của khách hàng mình nhận ra được một số điều khá "vô lý" trong logic. Ở dòng đầu tiên, mình thấy người dùng này không hề *đưa sản phẩm vào giỏ hàng* hay *bỏ sản phẩm ra khỏi giỏ*... Tuy nhiên, họ lại *đưa sản phẩm vào giỏ lần 2 sau khi đã lấy ra*!?

Do vậy, mình quyết định sử dụng 2 mệnh đề (cần thỏa mãn 1 trong 2) sau để tạo sự thống nhất trong việc xác định việc mua hàng:

- 1. Putting item into bag = TRUE **và** Taking item out of bag = FALSE
- 2. Putting item into bag = TRUE **và** Taking item out of bag = TRUE **và** Putting item into bag in the 2nd time = TRUE

**Ngoài ra**, các khách hàng không có đầy đủ 03 thông tin trên sẽ được loại khỏi dataset vì không thể xác định rõ việc mua hàng của họ.

Xác định đúng hành vi mua hàng của khách hàng cực kỳ quan trọng trong vì đây là phần dữ liệu dùng để trả lời cho khá nhiều câu hỏi nên mình rằng mong BTC sẽ không chỉ xem xét kết quả, mà còn nên đánh giá dựa trên logic của thí sinh!

### **2. Về việc sắp xếp lại các mặt hàng (câu số 11)**

Đầu tiên, dựa vào hai dataset liên quan đến **kệ hàng** và **sản phẩm** thì tất cả các mặt hàng đều được nằm đúng ở các kệ hàng theo đúng phân loại của mình!

Các kệ hàng đều có chiều cao tối đa là 180 cm nên việc phân tích có thể liên quan đến vị trí cao thấp của các mặt hàng trên kệ.

Một số thông tin cần quan tâm đến:
- Doanh thu của các quầy hàng (1)
- Sức mua của các quầy hàng (2)
- Thời lượng trung bình quan tâm đến sản phẩm, trên số lượt tương tác (3)
- Thông tin chi tiết về quầy hàng cụ thể (4)

Các quầy hàng có doanh thu thấp nhất:
| Shefl ID | Mô tả | Doanh thu |
| --- | --- | --- |
| 4 | Quầy gia vị | 32,642,800 |
| 6 | Quầy sách & đồ chơi | 29,129,000 |
| 0 | Quầy bánh kẹo | 24,424,700 |
| 3 | Quầy thực phẩm | 22,871,500 |

Các quầy hàng có sức mua thấp nhất:
| Shefl ID | Mô tả | Số lượng sản phẩm đã bán |
| --- | --- | --- |
| 4 | Quầy gia vị | 698 |
| 3 | Quầy thực phẩm | 522 |
| 5 | Quầy gia dụng | 371 |
| 6 | Quầy sách & đồ chơi | 282 |

**Suy luận**: Quầy gia dụng tuy có số lượt sản phẩm bán ra ít nhưng hợp lý bởi tần suất sử dụng thấp, đồng thời có doanh thu cao nhất. Còn quầy bánh kẹo, tuy có doanh thu chưa cao nhưng số lượng sản phẩm bán được lại là 969, nằm top 3 quầy hàng bán chạy (dẫn chứng câu 9). Do vậy, cần quan tâm hơn đến các quầy 3, 4 và 6.

Thời lượng quan tâm trung bình đến các quầy hàng 3, 4 và 6:
| Shefl ID | Mô tả | Thời lượng quan tâm trung bình (s) |
| --- | --- | --- |
| 3 | Quầy thực phẩm | 59.8 |
| 4 | Quầy gia vị | 58.0 |
| 6 | Quầy sách & đồ chơi | 39.5 |

**Suy luận**: Gia vị cũng như thực phẩm trên thực tế có tần suất sử dụng và có thể có mức độ quan tâm cao hơn hẳn các mặt hàng còn lại (đa phần khách đến cửa hàng đều nằm trong lứa tuổi trung và cao niên). Sách và đồ chơi nếu xét chung trên các danh mục mặt hàng hiện có thì tỏ ra quá khác biệt, khó có khả năng thu hút tiêu dùng so với khi được bày bán tại các nhà sách. Như vậy, một lần nữa chúng ta sẽ tập trung hơn vào hai quầy hàng số 3 và 4.

Tỷ lệ mua hàng trên số lượt tương tác (*số sản phẩm đã bán* (câu 9) chia cho *số lượt tương tác* (câu 8)):
| Shefl ID | Mô tả | Tỷ lệ |
| --- | --- | --- |
| **3** | **Quầy thực phẩm** | **40%** |
| **4** | **Quầy gia vị** | **37%** |
| 6 | Quầy sách & đồ chơi | 26% |

**Suy luận**: Tất cả các kệ hàng trong chi nhánh HomeMart này đều có chiều cao là 1.8 mét, lại được chia làm 03 loại vị trí đặt hàng là: *Thấp*, *Trung bình* và *Cao*. Chiều cao trung bình của người trưởng thành (trên 18 tuổi) tại Việt Nam là vào khoảng 1.53 đến 1.64 mét (số liệu thống kê năm 2010). Như vậy, vị trí đặt hàng *cao*, tương ứng với 1.2 mét trở lên sẽ là tối ưu nhất cho việc quan sát và lựa chọn, kế đến là các vị trí *trung bình* và *thấp*! Việc làm này một phần nào đó có thể cải thiện tỷ lệ mua và giúp tăng doanh thu cho quầy hàng.

Doanh thu và tình trạng hàng hóa tính theo vị trí của quầy **thực phẩm** (xem phần Answer 11):
| Vị trí | Số lượng trên kệ | Doanh thu |
|---|---|---|
| Trung bình | 18 | 15,762,800 |
| Thấp | 4 | 4,696,200 |
| Cao | 3 | 2,412,500 |

**Giải pháp**: Dễ thấy doanh thu cao nhất đến từ các sản phẩm ở vị trí *trung bình*; di chuyển thử nghiệm các sản phẩm dạng sợi ăn liền như mì tôm, hay nui (trọng lượng thấp) lên trên *cao*; và đưa các sản phẩm hiện đang ở trên *cao* xuống các vị trí thấp hơn. Các sản phẩm thường được đựng trong các chai, lọ thủy tinh như rượu, bia, nước ngọt thì cần có đánh giá để xem xét độ an toàn trước khi chuyển lên vị trí cao nhất.

Doanh thu và tình trạng hàng hóa tính theo vị trí của quầy **gia vị** (xem phần Answer 11):
| Vị trí | Số lượng trên kệ | Doanh thu |
|---|---|---|
| Cao | 7 | 24,342,000 |
| Thấp | 5 | 4,176,200 |
| Trung bình | 7 | 4,124,600 |

**Giải pháp**: Trong 12 sản phẩm bán được nhiều nhất của quầy hàng này, ngoài các sản phẩm được đặt sẵn ở vị trí *cao* thì có đến 03 sản phẩm hiện đang được đặt ở vị trí *thấp*, đó là các sản phẩm đường và mì chính. Với trọng lượng không quá lớn của mình, có thể dễ dàng chuyển lên vị trí cao nhất để tối ưu hơn nữa doanh thu cho hai loại mặt hàng này!

### **3. Về việc phân tích chiến lược quảng cáo, khuyến mại**

Sau khi phân tích (Answer 12) có thể nhận ra chiến dịch marketing không được áp dụng một mình mà luôn được áp dụng chung với sale. Do đó sau đây chỉ xét đến 03 loại: Áp dụng cả hai chiến lược, áp dụng sale và không áp dụng chiến lược nào.

Số lượng sản phẩm bán ra tại quầy bánh kẹo:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 171 |
| Marketing | 0 |
| Khuyến mãi | 407 |
| Không áp dụng | 391 |

Số lượng sản phẩm bán ra tại quầy hóa mỹ phẩm:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 375 |
| Marketing | 0 |
| Khuyến mãi | 288 |
| Không áp dụng | 495 |

Số lượng sản phẩm bán ra tại quầy sữa:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 167 |
| Marketing | 0 |
| Khuyến mãi | 62 |
| Không áp dụng | 545 |

Số lượng sản phẩm bán ra tại quầy thực phẩm:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 67 |
| Marketing | 0 |
| Khuyến mãi | 62 |
| Không áp dụng | 545 |

Số lượng sản phẩm bán ra tại quầy gia vị:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 75 |
| Marketing | 0 |
| Khuyến mãi | 463 |
| Không áp dụng | 160 |

Số lượng sản phẩm bán ra tại quầy gia dụng:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 187 |
| Marketing | 0 |
| Khuyến mãi | 104 |
| Không áp dụng | 0 |

Số lượng sản phẩm bán ra tại quầy sách và đồ chơi:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 0 |
| Marketing | 0 |
| Khuyến mãi | 0 |
| Không áp dụng | 282 |

Số lượng sản phẩm bán ra tại quầy đông lạnh:
| Chiến lược áp dụng | Số lượng hàng bán được |
|---|---|
| Cả hai | 169 |
| Marketing | 0 |
| Khuyến mãi | 69 |
| Không áp dụng | 908 |

### **4. Về hạn sử dụng của các mặt hàng**

Trong dataset về hành vi mua hàng, thời gian theo như mình phân tích được thì các dữ liệu được ghi nhận từ ngày 01/07/2024 đến 08/07/2024 (tức 8 ngày).

Tuy nhiên, bánh Custard - một sản phẩm bán khá chạy xét trên nhiều lứa tuổi, đứng đầu về doanh số khi xét trên các chiến lược quảng cáo hay khuyến mãi... lại có hạn sử dụng kết thúc vào ngày 13/04/2024, tức là cách thời điểm ghi nhận dữ liệu mua sắm gần 3 tháng. 

Tương tự cho một vài sản phẩm khác thuộc quầy hóa mỹ phẩm hay quầy sữa, với thời gian hết hạn tính đến thời điểm đang xét có lúc lên đến 7 tháng (Sữa bột Anmum Matern).

Bản thân mình cũng dự định sử dụng các thông tin này để phân tích cho việc sắp xếp lại các mặt hàng, nhưng sẽ không thể sử dụng vì có vẻ như phần dữ liệu này chưa đáng tin cậy cho lắm...